In [10]:
# Import the dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time

In [11]:
# Create a set of 2000 random latitudes and longitudes
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-90.000, high=90.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [12]:
# Get the nearest city using the citipy module.
from citipy import citipy

In [13]:
coordinates = list(lat_lngs)

In [14]:
# Create a list for holding the cities.
cities = []

# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)

        # Print the city count to confirm sufficient count.
len(cities)

81

In [15]:
# Perform an API call with the OpenWeatherMap.
# Import the requests library
import requests

# Import API key
from config import weather_api_key

In [16]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=26056fe164cee229334e682c90c6a5b3


In [17]:
# Retrieve the following information from the API call:

# Latitude and longitude
# Maximum temperature
# Percent humidity
# Percent cloudiness
# Wind speed
# Weather description (for example, clouds, fog, light rain, clear sky)

In [18]:
# Create an empty list to hold the weather data.
city_data = []

# Print the beginning of the logging
print("Beginning Data Retrieval      ")
print("------------------------------")

# Create counters
record_count = 1
set_count = 1

# Loop through all the cities in our list.
for i, city in enumerate(cities):
    
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)
        
    # Create endpoint URL with each city
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    # Add one to the record count
    record_count += 1
    
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather["weather"][0]["description"]
        
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Current Description": city_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval      
------------------------------
Processing Record 1 of Set 1 | grand gaube
Processing Record 2 of Set 1 | hithadhoo
Processing Record 3 of Set 1 | jamestown
Processing Record 4 of Set 1 | hermanus
Processing Record 5 of Set 1 | ushuaia
Processing Record 6 of Set 1 | punta arenas
Processing Record 7 of Set 1 | itaquyry
Processing Record 8 of Set 1 | santa cruz
Processing Record 9 of Set 1 | rurrenabaque
Processing Record 10 of Set 1 | dikson
Processing Record 11 of Set 1 | ponta delgada
Processing Record 12 of Set 1 | salym
Processing Record 13 of Set 1 | bambous virieux
Processing Record 14 of Set 1 | blagoyevo
Processing Record 15 of Set 1 | matsanga
City not found. Skipping...
Processing Record 16 of Set 1 | senanga
Processing Record 17 of Set 1 | victoria
Processing Record 18 of Set 1 | luanda
Processing Record 19 of Set 1 | saint-pierre
Processing Record 20 of Set 1 | ponta do sol
Processing Record 21 of Set 1 | bubaque
Processing Record 22 of Set 1 | 

In [19]:
# Add the data to a new DataFrame.
# Convert the array of dictionaries to a Pandas DataFrame
city_data_df = pd.DataFrame(city_data)
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,Grand Gaube,-20.0064,57.6608,68.38,74,7,3.00,MU,clear sky
1,Hithadhoo,-0.6000,73.0833,80.47,74,100,16.08,MV,overcast clouds
2,Jamestown,42.0970,-79.2353,78.03,49,20,8.05,US,few clouds
3,Hermanus,-34.4187,19.2345,39.79,72,2,6.64,ZA,clear sky
4,Ushuaia,-54.8000,-68.3000,37.35,98,100,4.50,AR,overcast clouds


In [29]:
# Export the DataFrame as a CSV file, and save it as WeatherPy_Database.csv in the Weather_Database folder.
# Create the output file CSV
output_data_file = "WeatherPy_Database.csv"

# Export the City_Data into a CSV
city_data_df.to_csv(output_data_file, index_label="City_ID")